In [1]:
import json
import pandas as pd

In [2]:
with open('all_courses_py.json', 'r') as file:
    all_courses = json.load(file)

all_courses[0]

{'course_code': 'ANAT 392',
 'course_title': 'Gross Anatomy of the Limbs and Trunk',
 'description': 'Lectures and laboratory sessions on the human gross and functional anatomy of the limbs and trunk. The course includes the study of predissected specimens. For credit only in the Department of Physical Therapy.',
 'prerequisites': [],
 'corequisites': [],
 'themes': []}

In [3]:
def gather_courses(course_code, all_courses, collected_courses=None):
    if collected_courses is None:
        collected_courses = set()
    
    for course in all_courses:
        if course['course_code'] == course_code and course_code not in collected_courses:
            collected_courses.add(course_code)
            
            # Gather prerequisites
            for prereq in course['prerequisites']:
                gather_courses(prereq, all_courses, collected_courses)
            
            # Gather corequisites
            for coreq in course['corequisites']:
                gather_courses(coreq, all_courses, collected_courses)
    
    return collected_courses

micb_courses = []
for course in all_courses:
    if course['course_code'].startswith('MICB'):
        course_codes = gather_courses(course['course_code'], all_courses)
        for code in course_codes:
            course_obj = next((c for c in all_courses if c['course_code'] == code), None)
            if course_obj and course_obj not in micb_courses:
                micb_courses.append(course_obj)

In [4]:
micb_courses[:2]

[{'course_code': 'CHEM 100',
  'course_title': 'Foundations of Chemistry',
  'description': 'Atomic and molecular properties, chemical reactions, bonding, nomenclature, kinetics, equilibrium processes, acids and bases, oxidation and reduction.  Intended for students who do not receive a satisfactory score on the UBC Chemistry Basic Skills Test, and need to establish a foundation of chemistry skills before proceeding to other first year chemistry courses. This course is not eligible for Credit/D/Fail grading. [3-0-1] Prerequisite: Not open to students with credit for Chemistry 12.  Permission of the department head.',
  'prerequisites': [],
  'corequisites': [],
  'themes': []},
 {'course_code': 'BIOL 111',
  'course_title': 'Introduction to Modern Biology',
  'description': 'Concepts fundamental to biological issues, such as the genetic basis of biological variation, evolution, infectious diseases, causes of cancer, population growth, and human effects on ecosystems.  Not open to stude

In [5]:
with open("micb_courses_dependancies_draft.json", "w") as file:
    json.dump(micb_courses, file, indent=4)

### Extracting Lecture/Lab

In [6]:
themes = pd.read_csv("micb_themes.csv")
themes.head(3)

,Course_Code,Data Science,Microbiology,Virology,Immunology,Science and Society,Climate,Human Health,Lecture,Lab Course,Notes
0,MICB_V 203,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,X,doesn't currently run
1,MICB_V 205,X,X,X,X,NaN,NaN,NaN,X,NaN,NaN
2,MICB_V 211,NaN,X,NaN,NaN,NaN,X,NaN,X,NaN,NaN


In [7]:
themes['course_code'] = themes['Course_Code'].str.replace('_V', '')
themes.drop(columns='Course_Code', inplace=True)
themes.head(3)

,Data Science,Microbiology,Virology,Immunology,Science and Society,Climate,Human Health,Lecture,Lab Course,Notes,course_code
0,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,X,doesn't currently run,MICB 203
1,X,X,X,X,NaN,NaN,NaN,X,NaN,NaN,MICB 205
2,NaN,X,NaN,NaN,NaN,X,NaN,X,NaN,NaN,MICB 211


In [8]:
theme_columns = ['Data Science', 'Microbiology', 'Virology', 'Immunology', 'Science and Society', 'Climate', 'Human Health']
themes['themes'] = themes[theme_columns].apply(lambda row: [col for col, value in row.items() if value == 'X'], axis=1)
themes['lab/lecture'] = themes.apply(lambda row: 'Lab' if row['Lab Course'] == 'X' else ('Lecture' if row['Lecture'] == 'X' else None), axis=1)

micb_themes = themes[['course_code', 'themes', 'lab/lecture']]
micb_themes.head(3)

,course_code,themes,lab/lecture
0,MICB 203,[Microbiology],Lab
1,MICB 205,"[Data Science, Microbiology, Virology, Immunol...",Lecture
2,MICB 211,"[Microbiology, Climate]",Lecture


In [9]:
with open('micb_courses_dependancies_draft.json', 'r') as file:
    all_micb_courses = json.load(file)

all_micb_courses[:5]

[{'course_code': 'CHEM 100',
  'course_title': 'Foundations of Chemistry',
  'description': 'Atomic and molecular properties, chemical reactions, bonding, nomenclature, kinetics, equilibrium processes, acids and bases, oxidation and reduction.  Intended for students who do not receive a satisfactory score on the UBC Chemistry Basic Skills Test, and need to establish a foundation of chemistry skills before proceeding to other first year chemistry courses. This course is not eligible for Credit/D/Fail grading. [3-0-1] Prerequisite: Not open to students with credit for Chemistry 12.  Permission of the department head.',
  'prerequisites': [],
  'corequisites': [],
  'themes': []},
 {'course_code': 'BIOL 111',
  'course_title': 'Introduction to Modern Biology',
  'description': 'Concepts fundamental to biological issues, such as the genetic basis of biological variation, evolution, infectious diseases, causes of cancer, population growth, and human effects on ecosystems.  Not open to stude

In [10]:
themes_dict = micb_themes.set_index('course_code').to_dict(orient='index')

# Update the list of dictionaries
for course in all_micb_courses:
    course_code = course['course_code']
    if course_code in themes_dict:
        course['themes'] = themes_dict[course_code]['themes']
        course['class_type'] = themes_dict[course_code]['lab/lecture']
    else:
        course['themes'] = []
        course['class_type'] = None

all_micb_courses[:3]

[{'course_code': 'CHEM 100',
  'course_title': 'Foundations of Chemistry',
  'description': 'Atomic and molecular properties, chemical reactions, bonding, nomenclature, kinetics, equilibrium processes, acids and bases, oxidation and reduction.  Intended for students who do not receive a satisfactory score on the UBC Chemistry Basic Skills Test, and need to establish a foundation of chemistry skills before proceeding to other first year chemistry courses. This course is not eligible for Credit/D/Fail grading. [3-0-1] Prerequisite: Not open to students with credit for Chemistry 12.  Permission of the department head.',
  'prerequisites': [],
  'corequisites': [],
  'themes': [],
  'class_type': None},
 {'course_code': 'BIOL 111',
  'course_title': 'Introduction to Modern Biology',
  'description': 'Concepts fundamental to biological issues, such as the genetic basis of biological variation, evolution, infectious diseases, causes of cancer, population growth, and human effects on ecosyste

In [11]:
with open("micb_courses_dependancies.json", "w") as file:
    json.dump(all_micb_courses, file, indent=4)